# Inverted Index

Associate a collection of terms (lexicon) with the documents that contain those terms.

The data structure is much more dense than a Document Term Matrix.


In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mac/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Collection of documents (corpus)
doc1 = "A professional business male, late 40s, 6 feet tall, slim build, well groomed, great personality, home owner, interests include the arts travel and all things good, Ringwood area, is seeking a genuine female of similar age or older, in same area or surrounds, for a meaningful long term rship. Looking forward to hearing from you all."
doc2 = "MALE LATE 50''s AUST Single, tall, prof. Interests: Music, theatre, dining, art, the beach and the environment. Seeking female with similar interests to share concerts, dining etc."
doc3 = "GENUINE AND HONEST Hi Im 44 with a good sense of humour, am romantic and love drives, fishing, camping and music. Love my 2 kids. Am looking for a lady with similar interests, aged between 38-45 for friendship/ possible relationship."

In [3]:
docs = [doc1, doc2, doc3]
docs

['A professional business male, late 40s, 6 feet tall, slim build, well groomed, great personality, home owner, interests include the arts travel and all things good, Ringwood area, is seeking a genuine female of similar age or older, in same area or surrounds, for a meaningful long term rship. Looking forward to hearing from you all.',
 "MALE LATE 50''s AUST Single, tall, prof. Interests: Music, theatre, dining, art, the beach and the environment. Seeking female with similar interests to share concerts, dining etc.",
 'GENUINE AND HONEST Hi Im 44 with a good sense of humour, am romantic and love drives, fishing, camping and music. Love my 2 kids. Am looking for a lady with similar interests, aged between 38-45 for friendship/ possible relationship.']

### Text processing 

In [4]:
def text_preprocessing(corpus) :
    print(corpus)
    # Tokenize the given document
    tokenized = word_tokenize(corpus)
    
    # Lower case all words
    tokenized = [word.lower() for word in tokenized]
    
    # Remove stopwords
    stopwords_en = set(stopwords.words("english"))
    stopwords_en = stopwords_en.union(set(punctuation))
    tokenized_without_sw = [word for word in tokenized if not word in stopwords_en]
    
    # Stemming 
    #porter = PorterStemmer()
    #tokenized_without_sw = [porter.stem(word) for word in tokenized_without_sw]
    
    # POS tagging 
    doc_tagged = pos_tag(tokenized_without_sw)
    
    # Lemmatizer
    wnl = WordNetLemmatizer()
    result = [wnl.lemmatize(word, pos=penn2morphy(tag[:2])) for word, tag in doc_tagged]
    
    return result

In [5]:
def penn2morphy(tag) : 
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[tag]
    except:
        return 'n' # if mapping isn't found, fall back to Noun.

In [6]:

docs_preprocessedV1 = []
docs_preprocessed = []
for doc in docs : 
    docs_preprocessedV1.append(text_preprocessing (doc))
for ls in docs_preprocessedV1:
    s = ' '.join(ls)
    docs_preprocessed.append(s)
    
docs_preprocessed

A professional business male, late 40s, 6 feet tall, slim build, well groomed, great personality, home owner, interests include the arts travel and all things good, Ringwood area, is seeking a genuine female of similar age or older, in same area or surrounds, for a meaningful long term rship. Looking forward to hearing from you all.
MALE LATE 50''s AUST Single, tall, prof. Interests: Music, theatre, dining, art, the beach and the environment. Seeking female with similar interests to share concerts, dining etc.
GENUINE AND HONEST Hi Im 44 with a good sense of humour, am romantic and love drives, fishing, camping and music. Love my 2 kids. Am looking for a lady with similar interests, aged between 38-45 for friendship/ possible relationship.


['professional business male late 40 6 foot tall slim build well groom great personality home owner interest include art travel thing good ringwood area seek genuine female similar age old area surround meaningful long term rship look forward hear',
 "male late 50 '' aust single tall prof interest music theatre din art beach environment seek female similar interest share concert din etc",
 'genuine honest hi im 44 good sense humour romantic love drive fish camp music love 2 kid look lady similar interest age 38-45 friendship/ possible relationship']

In [7]:
# Gather the set of all unique terms

unique_terms = {term for doc in docs_preprocessed for term in doc.split()}
unique_terms

{"''",
 '2',
 '38-45',
 '40',
 '44',
 '50',
 '6',
 'age',
 'area',
 'art',
 'aust',
 'beach',
 'build',
 'business',
 'camp',
 'concert',
 'din',
 'drive',
 'environment',
 'etc',
 'female',
 'fish',
 'foot',
 'forward',
 'friendship/',
 'genuine',
 'good',
 'great',
 'groom',
 'hear',
 'hi',
 'home',
 'honest',
 'humour',
 'im',
 'include',
 'interest',
 'kid',
 'lady',
 'late',
 'long',
 'look',
 'love',
 'male',
 'meaningful',
 'music',
 'old',
 'owner',
 'personality',
 'possible',
 'prof',
 'professional',
 'relationship',
 'ringwood',
 'romantic',
 'rship',
 'seek',
 'sense',
 'share',
 'similar',
 'single',
 'slim',
 'surround',
 'tall',
 'term',
 'theatre',
 'thing',
 'travel',
 'well'}

In [8]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability

inverted_index = {}

for i, doc in enumerate(docs_preprocessed):
    for term in doc.split():
        if term in inverted_index:
            inverted_index[term].add(i)
        else: inverted_index[term] = {i}

inverted_index

{'professional': {0},
 'business': {0},
 'male': {0, 1},
 'late': {0, 1},
 '40': {0},
 '6': {0},
 'foot': {0},
 'tall': {0, 1},
 'slim': {0},
 'build': {0},
 'well': {0},
 'groom': {0},
 'great': {0},
 'personality': {0},
 'home': {0},
 'owner': {0},
 'interest': {0, 1, 2},
 'include': {0},
 'art': {0, 1},
 'travel': {0},
 'thing': {0},
 'good': {0, 2},
 'ringwood': {0},
 'area': {0},
 'seek': {0, 1},
 'genuine': {0, 2},
 'female': {0, 1},
 'similar': {0, 1, 2},
 'age': {0, 2},
 'old': {0},
 'surround': {0},
 'meaningful': {0},
 'long': {0},
 'term': {0},
 'rship': {0},
 'look': {0, 2},
 'forward': {0},
 'hear': {0},
 '50': {1},
 "''": {1},
 'aust': {1},
 'single': {1},
 'prof': {1},
 'music': {1, 2},
 'theatre': {1},
 'din': {1},
 'beach': {1},
 'environment': {1},
 'share': {1},
 'concert': {1},
 'etc': {1},
 'honest': {2},
 'hi': {2},
 'im': {2},
 '44': {2},
 'sense': {2},
 'humour': {2},
 'romantic': {2},
 'love': {2},
 'drive': {2},
 'fish': {2},
 'camp': {2},
 '2': {2},
 'kid': {

In [9]:
# Now we can get posting lists for any term

In [10]:
posting_list = inverted_index['male']
posting_list

{0, 1}

In [11]:
# We can use the posting lists to locate the documents by ID (here just their ordering in the documents array)
# Think of this as a hash table, or a distributed hash table for much larger scenarios

In [12]:
# Notice now we can perform boolean operations on postings lists for Boolean search operations

In [13]:
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result

In [14]:
import numpy as np

query = "interest OR male"
query = query.split(" OR ")

pl_1 = list(inverted_index[query[0]])
pl_2 = list(inverted_index["".join(text_preprocessing(query[1]))])
results = or_postings(pl_1, pl_2)
for rs in results:
    print(docs[rs]+"\n")

male
A professional business male, late 40s, 6 feet tall, slim build, well groomed, great personality, home owner, interests include the arts travel and all things good, Ringwood area, is seeking a genuine female of similar age or older, in same area or surrounds, for a meaningful long term rship. Looking forward to hearing from you all.

MALE LATE 50''s AUST Single, tall, prof. Interests: Music, theatre, dining, art, the beach and the environment. Seeking female with similar interests to share concerts, dining etc.

GENUINE AND HONEST Hi Im 44 with a good sense of humour, am romantic and love drives, fishing, camping and music. Love my 2 kids. Am looking for a lady with similar interests, aged between 38-45 for friendship/ possible relationship.



In [15]:
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [16]:
query = "interest AND male"
query = query.split(" AND ")

pl_1 = list(inverted_index[query[0]])
pl_2 = list(inverted_index[query[1]])
results = and_postings(pl_1, pl_2)
for rs in results:
    print(docs[rs]+"\n")

A professional business male, late 40s, 6 feet tall, slim build, well groomed, great personality, home owner, interests include the arts travel and all things good, Ringwood area, is seeking a genuine female of similar age or older, in same area or surrounds, for a meaningful long term rship. Looking forward to hearing from you all.

MALE LATE 50''s AUST Single, tall, prof. Interests: Music, theatre, dining, art, the beach and the environment. Seeking female with similar interests to share concerts, dining etc.

